In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
lending = spark.read.csv('accepted_2007_to_2018Q4.csv.gz', inferSchema=True, header=True)
lending.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

## Decide on categories for features
loan_amount, funded_amnt, term, int_rate, grade, emp_length, home_ownership, annual_inc, addr_state?, fico_range_low (there's no point in using both FICO fields, they are close together and highly correlated), what else??

In [4]:
lending.select('loan_status').distinct().show() #shows different loan statuses

+--------------------+
|         loan_status|
+--------------------+
|          Fully Paid|
|             Default|
|                null|
|     In Grace Period|
|Does not meet the...|
|         Charged Off|
|            Oct-2015|
|  Late (31-120 days)|
|             Current|
|Does not meet the...|
|   Late (16-30 days)|
+--------------------+



Response variable is loan_status. We'll need to change to a binary with 'Default', 'Charged Off', and 'Does not meet the credit policy. Status:Charged Off' can be 1 for default and the others can be 0 for not defaulted. Null rows will need to be deleted.